In [ ]:
import os, random, shutil
from pathlib import Path

SPLIT_TREINO = 0.8   # 80 treino 20 teste
SEED = 42
EXTS = {".png", ".jpg", ".jpeg"}

BASE = Path.cwd()
IMAGENS = BASE / "imagens"
SRC = IMAGENS / "original"
DST_TRAIN = IMAGENS / "train"
DST_TEST  = IMAGENS / "test"

random.seed(SEED)

subdirs = [p for p in SRC.iterdir() if p.is_dir()]
if subdirs:
    classes = sorted([d.name for d in subdirs])
else:
    classes = ["_"]  # classe única; imagens diretamente em 'original'

# Garante diretórios de saída
for c in classes:
    (DST_TRAIN / c).mkdir(parents=True, exist_ok=True)
    (DST_TEST  / c).mkdir(parents=True, exist_ok=True)

def listar_imagens(pasta: Path):
    return sorted([p for p in pasta.glob("*") if p.suffix.lower() in EXTS])

def copiar(lista, destino: Path):
    for src_path in lista:
        if classes == ["_"]:
            dst_path = destino / src_path.name
        else:
            dst_path = destino / src_path.parent.name / src_path.name
        if not dst_path.exists():
            shutil.copy2(src_path, dst_path)

for c in classes:
    pasta_classe = SRC if c == "_" else (SRC / c)
    imgs = listar_imagens(pasta_classe)
    idx = list(range(len(imgs)))
    random.shuffle(idx)
    corte = max(1, int(SPLIT_TREINO * len(idx))) if len(idx) > 1 else len(idx)
    train_idx, test_idx = idx[:corte], idx[corte:]
    train_imgs = [imgs[i] for i in train_idx]
    test_imgs  = [imgs[i] for i in test_idx]
    copiar(train_imgs, DST_TRAIN)
    copiar(test_imgs,  DST_TEST)
print("total:", len(imgs), "train:", len(train_imgs), "test:", len(test_imgs))
